
# FPN Model Tasarımı (Sıfırdan İleri Seviyeye) + FPN Blok Kod Satır Satır + Entegrasyon

Bu notebook şunları **tek bir yerde** topluyor:

1) **FPN model nasıl tanımlanır?** (backbone→neck→head)  
2) **En baştan ileriye tasarım nasıl yapılır?** (tasarım checklist’i)  
3) **FPN blok tasarlanırken yol haritası** (boyut/kanal/stride)  
4) **FPN blok kodu** ve **her satırın ne yaptığı**  
5) **FPN’i geliştirme yolları** (P6/P7, PANet, BiFPN, weighted sum, attention, norm)  
6) **FPN’i modele entegre etme** (backbone’dan C2–C5 alma + head’e bağlama)



## 1) FPN modeli nasıl tanımlanır? (Backbone → Neck(FPN) → Head)

Bir detection/segmentation mimarisi genelde 3 parçadır:

### A) Backbone (özellik çıkarıcı)
- CNN (ResNet, CSPDarknet, EfficientNet, ConvNeXt…)
- Çıktı: farklı stride’larda feature map’ler
  - **C2** (stride 4), **C3** (stride 8), **C4** (stride 16), **C5** (stride 32)

### B) Neck (FPN)
- Amaç: C2–C5’i birleştirip **P2–P5** üretmek
- Top-down upsample + lateral 1×1 + element-wise sum + (opsiyonel) 3×3 smooth

### C) Head (görev başlığı)
- Detection head: her P seviyesinde sınıflandırma + bbox regresyon
- Segmentation head: genelde yüksek çözünürlük (P2/P3) ağırlıklı kullanır



## 2) En baştan ileriye FPN tasarım checklist’i

### 2.1 Hangi seviyeleri kullanacaksın?
- Klasik: **C2–C5** → **P2–P5**
- Hafif: **C3–C5** → **P3–P5**

### 2.2 Stride haritasını netleştir
- C2: H/4 × W/4
- C3: H/8 × W/8
- C4: H/16 × W/16
- C5: H/32 × W/32

### 2.3 Output channel standardı
- Klasik FPN: tüm P seviyeleri **out_channels = 256**

### 2.4 Fusion tipi
- Baseline: **sum** (element-wise)
- Alternatif: concat + conv (daha pahalı)

### 2.5 Upsample modu
- `nearest`: hızlı
- `bilinear`: daha yumuşak (genelde `align_corners=False`)

### 2.6 Smooth conv (3×3)
- Klasik FPN’de vardır (upsample artefact azaltır)

### 2.7 Ek seviye (opsiyonel)
- RetinaNet tarzı: **P6/P7**



## 3) FPN blok tasarlanırken yol haritası (en kritik 5 kural)

1) **Toplama için shape eşit olacak**: `(B,C,H,W)` aynı olmalı → gerekirse upsample.  
2) **Lateral 1×1 conv şart**: kanal hizalama (Ck kanalları farklı).  
3) **Top-down sırayı bozma**: P5→P4→P3→P2.  
4) **Bu backward değil**: forward grafiğinde ek bir yol.  
5) **Head’e Ck değil Pk ver**: FPN’nin çıktıları P seviyeleridir.



## 4) Baseline FPN bloğu (PyTorch)

Aşağıdaki blok:
- `in_channels_list = [C2_in, C3_in, C4_in, C5_in]`
- çıktı: `[P2, P3, P4, P5]` (hepsi `out_channels` kanal)

Sonraki bölümde satır satır açıklama var.


In [1]:

import torch
import torch.nn as nn
import torch.nn.functional as F

class FPN(nn.Module):
    # Klasik FPN: Top-Down + Lateral + Sum + Smooth
    # feats: [C2, C3, C4, C5]
    # return: [P2, P3, P4, P5]
    def __init__(self, in_channels_list, out_channels=256, upsample_mode="nearest"):
        super().__init__()
        assert len(in_channels_list) == 4, "Bu implementasyon C2..C5 (4 seviye) bekliyor."
        self.upsample_mode = upsample_mode

        # 1) Lateral 1x1: Ck -> out_channels (kanal hizalama)
        self.lateral = nn.ModuleList([
            nn.Conv2d(cin, out_channels, kernel_size=1, bias=False)
            for cin in in_channels_list
        ])

        # 2) Smooth 3x3: toplama sonrası düzeltme
        self.smooth = nn.ModuleList([
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False)
            for _ in in_channels_list
        ])

    def forward(self, feats):
        C2, C3, C4, C5 = feats

        # Lateral projeksiyonlar
        L2 = self.lateral[0](C2)
        L3 = self.lateral[1](C3)
        L4 = self.lateral[2](C4)
        L5 = self.lateral[3](C5)

        # Top-down + sum (FPN kalbi)
        P5 = L5
        P4 = L4 + F.interpolate(P5, size=L4.shape[-2:], mode=self.upsample_mode)
        P3 = L3 + F.interpolate(P4, size=L3.shape[-2:], mode=self.upsample_mode)
        P2 = L2 + F.interpolate(P3, size=L2.shape[-2:], mode=self.upsample_mode)

        # Smooth (klasik FPN’de var)
        P2 = self.smooth[0](P2)
        P3 = self.smooth[1](P3)
        P4 = self.smooth[2](P4)
        P5 = self.smooth[3](P5)

        return [P2, P3, P4, P5]

# hızlı shape testi
B = 2
C2 = torch.randn(B, 256, 128, 128)
C3 = torch.randn(B, 512, 64, 64)
C4 = torch.randn(B, 1024, 32, 32)
C5 = torch.randn(B, 2048, 16, 16)

fpn = FPN([256, 512, 1024, 2048], out_channels=256)
P2, P3, P4, P5 = fpn([C2, C3, C4, C5])
[P.shape for P in (P2, P3, P4, P5)]


[torch.Size([2, 256, 128, 128]),
 torch.Size([2, 256, 64, 64]),
 torch.Size([2, 256, 32, 32]),
 torch.Size([2, 256, 16, 16])]


## 5) FPN blok satır satır: bu satır ne işe yarıyor?

### `assert len(in_channels_list) == 4`
- Bu sürüm 4 seviye bekliyor (C2–C5). Farklı sayıda seviye vereceksen kodu genelleştirmen gerekir.

### `self.lateral = ModuleList(Conv1×1...)`
- C2..C5 kanal sayıları farklı → **hepsini 256’ya indiriyoruz** (veya out_channels kaçsa).
- Sum yapmanın ön koşulu: kanal eşitliği.

### `self.smooth = ModuleList(Conv3×3...)`
- Upsample + sum sonrası **aliasing/artefact** olabilir.
- 3×3 conv ile daha stabil P seviyeleri çıkar.

### `Lk = lateral(Ck)`
- Ck (ham backbone) → Lk (kanalı eşitlenmiş)

### `P5 = L5`
- En üst seviye doğrudan L5 (en semantik seviye).

### `P4 = L4 + interpolate(P5, size=L4)`
- P5 küçük → P4 boyutuna büyüt → **aynı koordinatta element-wise sum**.
- Semantik (P5) + detay (L4).

### Aynı mantık P3 ve P2 için.



## 6) FPN blok modele nasıl entegre edilir? (net 3 adım)

### Adım-1: Backbone’dan C2–C5 çıkar
- Ya backbone’un forward’ında `return [C2,C3,C4,C5]` yap
- Ya da (torchvision varsa) feature extractor kullan

### Adım-2: FPN’yi neck olarak çağır
- `P2..P5 = fpn([C2,C3,C4,C5])`

### Adım-3: Head’i P seviyelerine bağla
- Detection head: her P üzerinde aynı küçük conv stack



### 6.1 Çalışan demo: Dummy Backbone + FPN + Dummy Head

Bu gerçek detector değil. Entegrasyon mantığını **çalışan** şekilde gösteriyor.


In [2]:

import torch
import torch.nn as nn
import torch.nn.functional as F

class DummyBackbone(nn.Module):
    # Sadece demo: input'u kademeli downsample edip C2..C5 üretir.
    def __init__(self):
        super().__init__()
        self.c2 = nn.Sequential(nn.Conv2d(3, 256, 3, stride=2, padding=1), nn.ReLU())     # /2
        self.c3 = nn.Sequential(nn.Conv2d(256, 512, 3, stride=2, padding=1), nn.ReLU())   # /4
        self.c4 = nn.Sequential(nn.Conv2d(512, 1024, 3, stride=2, padding=1), nn.ReLU())  # /8
        self.c5 = nn.Sequential(nn.Conv2d(1024, 2048, 3, stride=2, padding=1), nn.ReLU()) # /16

    def forward(self, x):
        C2 = self.c2(x)
        C3 = self.c3(C2)
        C4 = self.c4(C3)
        C5 = self.c5(C4)
        return [C2, C3, C4, C5]

class DummyHead(nn.Module):
    # Demo: her P seviyesini pool edip linear ile sınıf skoru üretir.
    def __init__(self, in_ch=256, num_classes=5):
        super().__init__()
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(in_ch, num_classes)

    def forward(self, pyramids):
        logits = []
        for P in pyramids:
            v = self.pool(P).flatten(1)  # (B,C)
            logits.append(self.fc(v))    # (B,num_classes)
        return sum(logits) / len(logits)

class ModelWithFPN(nn.Module):
    def __init__(self, num_classes=5):
        super().__init__()
        self.backbone = DummyBackbone()
        self.fpn = FPN([256, 512, 1024, 2048], out_channels=256)
        self.head = DummyHead(in_ch=256, num_classes=num_classes)

    def forward(self, x):
        feats = self.backbone(x)       # C2..C5
        pyr = self.fpn(feats)          # P2..P5
        out = self.head(pyr)           # demo logits
        return out

m = ModelWithFPN(num_classes=5)
x = torch.randn(4, 3, 256, 256)
y = m(x)
y.shape


torch.Size([4, 5])


## 7) FPN nasıl geliştirilir? (upgrade seçenekleri)

### 7.1 P6/P7 ekle (RetinaNet gibi)
- P6 genelde `Conv(stride=2)` ile P5’ten üretilir
- P7 genelde P6’dan bir stride=2 daha

### 7.2 PANet (Bottom-up path augmentation)
- FPN top-down yapar.
- PANet, alt seviyeden üste **ek bir yol** kurar (özellikle localization tarafı).

### 7.3 BiFPN (EfficientDet)
- Fusion’da learnable weight: `w1*X + w2*Y`
- Birkaç kez tekrarlanan BiFPN blokları

### 7.4 Weighted sum (FPN içinde küçük upgrade)
- `P4 = α*L4 + β*Up(P5)` gibi ağırlıklı toplama (α,β öğrenilir)

### 7.5 Norm/Act ekle
- Lateral ve smooth katmanlarına GN/BN + SiLU/ ReLU eklemek
- Stabilite artabilir

### 7.6 Attention ekle
- P seviyelerine SE/CBAM gibi attention → maliyet artar ama bazı task’larda iyi



## 8) Hata ayıklama checklist’i

- **RuntimeError: size mismatch** → upsample `size=` yanlış, stride haritası yanlış
- **Channel mismatch** → lateral 1×1 conv yok/yanlış, out_channels tutmuyor
- **P seviyeleri karışık** → top-down sırası ters yazılmış
- **Küçük nesne kötü** → P2 yok / stride haritası yanlış / backbone C2 çok erken kesilmiş
